# Thunor Tutorial

Thunor is a Python package for managing and viewing multi-timepoint high throughput screen data. It utilizes the drug-induced proliferation rate (DIP rate) metric, which is a dynamic measure of drug response.

The Python package also has a web interface available, called Thunor Web.

## [Ignore] Set up code for documentation

In [ ]:
# Temporary hack, see https://github.com/poliastro/poliastro/issues/281
from IPython.display import HTML
HTML('<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.1.10/require.min.js"></script>')

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))

## Start Jupyter Notebook

Run jupyter notebook with the following argument:

    jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
    
The data rate limit needs to be increased or `init_notebook_mode()` throws an error.

## Load a file

First, specify a file to load. This can be done as a string filename, but here we load an example dataset from the thunor package.

In [ ]:
import pkg_resources
hts007_file = pkg_resources.resource_filename('thunor', 'testdata/hts007.h5')

Load the file using `read_hdf` (for HDF5 files), `read_vanderbilt_hts` (for CSV files), or another appropriate reader.

In [ ]:
from thunor.io import read_hdf
hts007 = read_hdf(hts007_file)

We'll just use a subset of the drugs, to make the plots manageable.

In [ ]:
hts007r = hts007.filter(drugs=['cediranib', 'everolimus', 'paclitaxel'])

In [ ]:
hts007r.drugs

In [ ]:
hts007r.cell_lines

## Calculate DIP rates and parameters

These two operations can be done in two lines of code (plus imports).

In [ ]:
from thunor.dip import dip_rates
from thunor.curve_fit import fit_params

ctrl_dip_data, expt_dip_data = dip_rates(hts007r)
fp = fit_params(ctrl_dip_data, expt_dip_data)

## Setting up plots

Each of the `plot_X` functions returns a plotly `Figure` object which can be visualised in a number of ways. Here, we use the offline `iplot` function, which generates a plot for use with Jupyter notebook. We could also generate plots using the `plot` function in standalone HTML files. See the [plotly documentation](https://plot.ly/python/offline/) for more information on the latter approach. 

In [ ]:
from thunor.plots import plot_drc, plot_drc_params, plot_time_course, plot_ctrl_dip_by_plate, plot_plate_map
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode()

## Plot Types

### Plot DIP rate curves

In [ ]:
iplot(plot_drc(fp))

### Plot DIP parameters

In [ ]:
iplot(plot_drc_params(fp, 'auc'))

### Filtering fit params

The `fp` object is a pandas data frame, so we can filter it before plotting. Some examples:

In [ ]:
fit_params_bt20_pac = fp[fp.index.isin(['BT20'], level='cell_line') & \
                         fp.index.isin(['paclitaxel'], level='drug')]

iplot(plot_drc(fit_params_bt20_pac))

### Plot time course

Time course plot for paclitaxel on BT20 cells:

In [ ]:
iplot(plot_time_course(
    hts007.filter(drugs=['paclitaxel'], cell_lines=['BT20'])
))

### Quality control check: plot DIP rate ranges by cell line and plate (box plot)

In [ ]:
iplot(plot_ctrl_dip_by_plate(ctrl_dip_data))

### Quality control check: plot DIP rate as a plate heat map

In [ ]:
plate_data = hts007.plate('HTS007_149-28A', include_dip_rates=True)

In [ ]:
iplot(plot_plate_map(plate_data, color_by='dip_rates'))